## Two python techniques

1. dictionaries
2. lambda functions

## Grouping and Summarizing

In [ ]:
import pandas as pd
import  numpy as np

Our first pass will be with our old friends the penguins.

In [ ]:
penguins = pd.read_csv("data/penguins-raw.csv")

## Some basic data cleaning

1.  Data types

In [ ]:
penguins.dtypes

2. Focus on Species, Island, Sex, Culmen Length/Depth, Flipper Length, Body Mass.

In [ ]:
focus = ['Species','Island','Sex','Culmen Length (mm)','Culmen Depth (mm)','Flipper Length (mm)', 'Body Mass (g)']
simplified = penguins[focus]

3. Clean up column names

In [ ]:
edited_columns = ['species','island','sex','culmen_length', 'culmen_depth','flipper_length','body_mass']
simplified.columns = edited_columns

4. Simplify factor names. (Note use of dictionary)


In [ ]:
species = simplified['species'].unique()
simple_species_dict={x:x.split(' ')[0].lower() for x in species}
simplified['species'].map(simple_species_dict)

5. Remaking a column (watch out!)

In [ ]:
#simplified['species'] = simplified['species'].map(simple_species_dict)

Old option: use .loc.

In [ ]:
#simplified.loc[:,'species'] = simplified['species'].map(simple_species_dict)

Newer option: use .assign().  Notice that .assign() *returns a dataframe.*


In [ ]:
simplified = simplified.assign(species = lambda x: x['species'].map(simple_species_dict))

Fix some other factor variables:

In [ ]:
simplified  = simplified.assign(island = lambda x: x.island.str.lower())
simplified = simplified.assign(sex = lambda x: x['sex'].str.lower())

5.  Standardize the variables - column by column

In [ ]:
#simplified = simplified.assign(culmen_length_std = lambda x: (x.culmen_length-x.culmen_length.mean())/x.culmen_length.std())

or make a standardization function. (note use of **)

In [ ]:
def standardize(x):
    return (x-x.mean())/x.std()
simplified = simplified.assign(
    **{i+'_std':(lambda x: standardize(x[i])) for i in simplified.columns[3:]}
)

6.  Missing Values

In [ ]:
simplified.isna().sum()
simplified = simplified.dropna(axis=0)

7.  Grouping

Grouping combines with aggregation.

In [ ]:
numerical_variables = ['culmen_length','culmen_depth','flipper_length','body_mass']
by_sex_mean = simplified[['sex']+numerical_variables].groupby('sex').mean()

Alternatively one can use .agg

In [ ]:
numerical_variables = ['culmen_length','culmen_depth','flipper_length','body_mass']
by_sex = simplified[['sex']+numerical_variables].groupby('sex').agg('mean')

And then get multiple aggregations.

In [ ]:
by_sex = simplified[['sex']+numerical_variables].groupby('sex').agg(['count','mean','std'])

To access individual elements, use tuples as names.

In [ ]:
by_sex.loc[:,('culmen_depth','mean')]

One can also group on multiple factors.

In [ ]:
by_sex_and_species = simplified[['sex','species']+numerical_variables].groupby(['sex','species']).mean()
females_by_species = by_sex_and_species.loc[('female',)]
males_by_species = by_sex_and_species.loc[('male',)]

8.  Pivot tables

In [ ]:
expanded = by_sex_and_species.reset_index()
expanded.pivot(index='sex',columns='species',values='culmen_length')

In [ ]:
pd.pivot_table(simplified,values='culmen_length',index='sex',columns='species',aggfunc='mean')